In [8]:
from google.colab import files

uploaded = files.upload()

Saving rf_ton_iot_pipeline.joblib to rf_ton_iot_pipeline (1).joblib


In [9]:
from google.colab import files

uploaded = files.upload()

Saving xgb_ton_iot_pipeline.joblib to xgb_ton_iot_pipeline (1).joblib


In [14]:
import os
os.environ["NGROK_AUTHTOKEN"] = "32NJXIDUDoqLgQWeeRkM9pxZgB1_VdVWX8zAz9ip7U45k9VK"


In [ ]:
# ===== MEDSECURE AI — ONE-CELL FASTAPI FOR COLAB =====
# Installs + API + runner in one place.

# 1) Install dependencies
!pip -q install fastapi uvicorn nest_asyncio pyngrok joblib pandas numpy shap xgboost scikit-learn

import os, io, json, time, base64, joblib, pandas as pd, numpy as np
import matplotlib.pyplot as plt
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio, uvicorn
from pyngrok import ngrok
from typing import List, Optional, Any

# 2) Model file discovery (be tolerant to upload names)
CANDIDATES = [
    "rf_ton_iot_pipeline.joblib",
    "xgb_ton_iot_pipeline.joblib",
    "rf.joblib", "xgb.joblib",
    "rf_pipeline.joblib", "xgb_pipeline.joblib"
]

def find_first_existing(names: List[str]) -> Optional[str]:
    for n in names:
        if os.path.exists(n):
            return n
    # also try any uploaded joblib
    rf_guess, xgb_guess = None, None
    for f in os.listdir():
        if f.lower().endswith(".joblib"):
            fl = f.lower()
            if any(k in fl for k in ["rf", "randomforest", "random_forest"]) and rf_guess is None:
                rf_guess = f
            if any(k in fl for k in ["xgb", "xgboost"]) and xgb_guess is None:
                xgb_guess = f
    # fallback check
    for cand in [rf_guess, xgb_guess]:
        if cand and os.path.exists(cand):
            names.append(cand)
    for n in names:
        if os.path.exists(n):
            return n
    return None

RF_PATH  = find_first_existing(CANDIDATES[:3] + ["rf_ton_iot_pipeline.joblib"])
XGB_PATH = find_first_existing(CANDIDATES[3:] + ["xgb_ton_iot_pipeline.joblib"])

if not RF_PATH or not XGB_PATH:
    raise FileNotFoundError(
        "Could not find both RF and XGB .joblib files in the current directory. "
        "Upload them via files.upload() (left panel > Files, or from code) or set RF_PATH/XGB_PATH manually."
    )

rf_model  = joblib.load(RF_PATH)
xgb_model = joblib.load(XGB_PATH)
print(f"Loaded models:\n - RF : {RF_PATH}\n - XGB: {XGB_PATH}")

# 3) Infer required feature columns from pipelines
def get_preprocessor(model: Any):
    for attr in ("preprocessor_", "preprocess_", "preprocessor", "transformer_", "column_transformer_", "column_transformer"):
        pre = getattr(model, attr, None)
        if pre is not None:
            return pre
    steps = getattr(model, "steps", None)
    if steps:
        for name, step in steps:
            if any(k in name.lower() for k in ["preprocess", "preprocessor", "column"]):
                return step
    return None

def required_cols(model: Any) -> List[str]:
    pre = get_preprocessor(model)
    if pre is None:
        fn = getattr(model, "feature_names_in_", None)
        return list(fn) if fn is not None else []
    if hasattr(pre, "feature_names_in_"):
        return list(pre.feature_names_in_)
    if hasattr(pre, "transformers_"):
        cols = []
        for _, _, cols_or_idx in pre.transformers_:
            if isinstance(cols_or_idx, list):
                cols.extend([str(c) for c in cols_or_idx])
        # de-dup keep order
        return list(dict.fromkeys(cols))
    return []

RF_COLS  = required_cols(rf_model)
XGB_COLS = required_cols(xgb_model)
print(f"RF expected columns: {len(RF_COLS)}")
print(f"XGB expected columns: {len(XGB_COLS)}")

def align_to_schema(payload: dict, cols: List[str]) -> pd.DataFrame:
    X = pd.DataFrame([payload])
    return X.reindex(columns=cols, fill_value=np.nan) if cols else X

# 4) FastAPI
THRESHOLD = 0.5
LOG_PATH  = "predictions_log.jsonl"
app = FastAPI(title="MedSecure AI – Inference API", version="1.0")

class TrafficEvent(BaseModel):
    payload: dict

def predict_one(model_name: str, payload: dict):
    model = rf_model if model_name.lower() == "rf" else xgb_model
    cols  = RF_COLS if model_name.lower() == "rf" else XGB_COLS
    X     = align_to_schema(payload, cols)
    yprob = getattr(model, "predict_proba")(X)[:, 1]
    yhat  = (yprob >= THRESHOLD).astype(int)
    return float(yprob[0]), int(yhat[0])

@app.get("/health")
def health():
    return {"status": "ok", "rf_cols": len(RF_COLS), "xgb_cols": len(XGB_COLS)}

@app.get("/schema")
def schema():
    return {"rf_columns": RF_COLS, "xgb_columns": XGB_COLS}

@app.post("/predict")
def predict(evt: TrafficEvent):
    rf_p, rf_y = predict_one("rf", evt.payload)
    xg_p, xg_y = predict_one("xgb", evt.payload)
    votes = rf_y + xg_y
    final = 1 if votes >= 1 else 0  # conservative OR
    probs = {"rf": rf_p, "xgb": xg_p}
    decision = "attack" if final == 1 else "normal"
    rec = {"ts": time.time(), "payload_keys": list(evt.payload.keys()), "probs": probs, "final_label": final}
    with open(LOG_PATH, "a") as f:
        f.write(json.dumps(rec) + "\n")
    return {"probs": probs, "final_label": final, "decision": decision}

@app.post("/explain")
def explain(evt: TrafficEvent, model: str = "rf", top_k: int = 10):
    model = model.lower()
    mdl   = rf_model if model == "rf" else xgb_model
    cols  = RF_COLS  if model == "rf" else XGB_COLS
    X     = align_to_schema(evt.payload, cols)

    import shap
    # Model-agnostic explainer using prediction function
    explainer = shap.Explainer(mdl.predict, X, feature_names=X.columns.tolist())
    shap_vals = explainer(X)

    vals = pd.DataFrame({"feature": X.columns, "value": np.abs(shap_vals.values[0])})
    top = vals.sort_values("value", ascending=False).head(top_k)

    plt.figure()
    top.set_index("feature")["value"].plot(kind="barh")
    plt.title(f"Top {top_k} SHAP Features ({model})")
    plt.xlabel("SHAP value (impact)")
    plt.tight_layout()
    buf = io.BytesIO(); plt.savefig(buf, format="png"); plt.close()
    img_b64 = base64.b64encode(buf.getvalue()).decode()

    return {
        "model": model,
        "top_features": top.to_dict(orient="records"),
        "shap_plot_base64": img_b64
    }

# 5) Run server + ngrok (expects NGROK_AUTHTOKEN env var)
nest_asyncio.apply()

token = os.environ.get("NGROK_AUTHTOKEN")
if token:
    ngrok.set_auth_token(token)
else:
    print("WARNING: Set your ngrok token once per runtime:")
    print('  import os; os.environ["NGROK_AUTHTOKEN"] = "PASTE_TOKEN_HERE"')

tunnel = ngrok.connect(8000, "http")
print("Public URL:", tunnel.public_url, " (open /docs for Swagger UI)")
uvicorn.run(app, host="0.0.0.0", port=8000)



Loaded RF: rf_ton_iot_pipeline.joblib
Loaded XGB: DISABLED (XGBoostError: [12:04:38] ../src/common/json.cc:458: Expecting: """, got: "76 ", around character position: 1
    {L\0\0\0\0\0\0\0
    ^~~~~~~~~
Stack trace:
  [bt] (0) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0xcb05d) [0x786d18ecb05d]
  [bt] (1) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0xccd95) [0x786d18eccd95]
  [bt] (2) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0xd1d02) [0x786d18ed1d02]
  [bt] (3) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0xd309d) [0x786d18ed309d]
  [bt] (4) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x1bfb3d) [0x786d18fbfb3d]
  [bt] (5) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(XGBoosterUnserializeFromBuffer+0x5c) [0x786d18e98bcc]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x786d74e02e2e]
  [bt] (7) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x786

INFO:     Started server process [30753]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://a5ddf6d9d3ed.ngrok-free.app  (open /docs)
INFO:     180.233.124.109:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     180.233.124.109:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     180.233.124.109:0 - "GET /dpcs HTTP/1.1" 404 Not Found
INFO:     180.233.124.109:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     180.233.124.109:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     180.233.124.109:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     180.233.124.109:0 - "POST /explain?model=rf&top_k=10 HTTP/1.1" 200 OK
INFO:     180.233.124.109:0 - "GET /schema HTTP/1.1" 200 OK
INFO:     180.233.124.109:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     180.233.124.109:0 - "POST /predict HTTP/1.1" 422 Unprocessable Entity
INFO:     180.233.124.109:0 - "POST /predict HTTP/1.1" 200 OK
